# Preprocessing the files from database into subtitles folder

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import zipfile
import io

In [2]:
conn = sqlite3.connect('eng_subtitles_database.db')

# SQL query
query = "SELECT num, name, content FROM zipfiles"

In [3]:
df = pd.read_sql_query(query, conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [15]:
import re
def clean_filename(filename):
    # Remove extension and language/version info
    filename = re.sub(r"\.\w+$", "", filename)  # Removes .1cd
    # Replace dots and underscores with spaces
    filename = re.sub(r"[\._]", " ", filename)
    # Remove year and parentheses
    filename = re.sub(r"\(\d{4}\)", "", filename)
    # Remove eng
    filename = re.sub(r" eng", " ", filename)
    
    return filename.strip().lower()

df["name"] = df["name"].apply(clean_filename)

In [18]:
df['name'][40000]

'nature s35 e12 yosemite'

In [19]:
#saving ids and name for keyword retrieval
df = df.drop(columns=['content'])
df.to_csv('subtiles_id_names.csv', index=False)

In [5]:
def extract_and_decode(binary_data):
    '''
    Function to extract content from binary zip file and decode it
    '''
    try:
        # Create a BytesIO object from the binary data
        with io.BytesIO(binary_data) as f:
           
            with zipfile.ZipFile(f, 'r') as zip_file:
            
                subtitle_content = zip_file.read(zip_file.namelist()[0])
                
        # Decode the content using 'latin-1' encoding
        decoded_content = subtitle_content.decode('latin-1')
        return decoded_content
        
    except Exception as e:
        return f"Extraction failed with error: {str(e)}"

In [6]:
#reading an example file from the database

file_content = df['content'][1]
file_content[:500]

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x99V\x12o\xb0\xc2g\x0f\x00\x00Z$\x00\x008\x00\x00\x00Here Comes the Grump - Ep. 9 - Joltin_ Jack-in boxia.srtuZ]s\x1b7\x96}g\x15\xff\x03\xf4\xe4\x97\x96\xa6\xbf\xd0\x1f\xaeT\\v\xb2\xb15\xc9lf\xc6N\xa9\xfc\xd8"A\xb1Gd7\xb7\xbb)\x86\xff~\xcf\xb9h@hm\xb6*e\xc5\xa4ppqq\xee\xb9\x1fp\xb2^\xc5\xf1{\xfc\x97\xd6Q\xa6ku{\xfb\xa3\xb2\x1fdi\x14\xe7\xd5z\xf5q\x7f\xa3\xbe\xed\xcd`\xde\x8d\xea\x9fC\xdbm\xcc8\xaeW?7\x97N5\xddV}3\xc3pU\x97v\xda\xabio\xd6\xab\xf5*u\x90@H\xf3*\x84\xcc\xa32\xaf\xd7\xabO\x87\xbe\xef\xccU\xfdf\x7f\x8c\xfda\xdb\x9a\x01\xa8\xa7\xa1\x9f\xccfj\xbb\'\x82\x1d\xef\x08\x97y\xb8:*\xe28\x80\xcb\xd3(O\xf2\xf5\xea\xf7}d\xcd\x88\x14\x80\x9fo\xd6\xab/0Wm\xfa\xa3\x19\x89\xa3>\x0f\xe7\xe3I\xc0r\x07\xc6\xb5i\x19\x82\xe9\xa8\xac\xf5z\xf5\xdd48\xf0G\x9c\x8cX\xa3:\xb4\xcfF\xedy\xf8\xa7^\xec\xea\xd7\xabq\xdf\xf7\x13\xcc>\x1a\xb5m\x86iT\xcd\xa4\xfa\xf3\xa0\x1e\x9b\x03O&;i\xbfS\x11\xe5Uh\xb6N\xa2:I\xd6\xab\x07s8D\xaa\xdd\xd1\xc4+\xfeh\xbbg\xf5\xcc-\x1e

In [7]:
print(extract_and_decode(file_content)[:1000])

1
00:00:29,359 --> 00:00:32,048
Ah! There's Princess
Dawn and Terry with the

2
00:00:32,248 --> 00:00:34,749
Blooney Looney soldiers
protecting them.

3
00:00:39,600 --> 00:00:42,414
Oh, Terry, look!
Here comes the Grump.

4
00:00:42,427 --> 00:00:45,795
Yeah! And looks like he's going to
shoot some darts at our balloon.

5
00:00:46,480 --> 00:00:51,911
Well, if they think king blooney's gas
bag guards can protect them from me.

6
00:00:52,111 --> 00:00:54,195
We got another thing coming.

7
00:00:54,559 --> 00:00:57,428
Okay! When I get the signal,
we'll make our move.

8
00:00:58,239 --> 00:00:59,868
This will fix him.

9
00:01:02,480 --> 00:01:04,224
Okay.
Everybody up!

10
00:01:06,400 --> 00:01:07,843
Oh, Terry, I'm frightened.

11
00:01:08,043 --> 00:01:10,208
Isn't there some way
we can fight back?

12
00:01:10,880 --> 00:01:14,414
I'd better do something,
and quick. Here he comes again.

13
00:01:15,040 --> 00:01:18,161


In [8]:
#applying the extract and decode function to whole database to see that content



%time df["file_content"] = df['content'].apply(extract_and_decode)

CPU times: total: 14.5 s
Wall time: 15min 7s


In [10]:
df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [12]:
import os
from tqdm import tqdm

def save_content_to_txt(row):
    """
    Function to save content from a dataframe row to a .txt file.
    """
    try:
        file_name = f"subtitles/{row['name']}.txt"
        content = row['file_content']
        
        # Write the content to file
        with open(file_name, 'w', encoding='latin-1') as file:
            file.write(row['name'])
            file.write(content)
        return file_name
    except Exception as e:
        return f"Error saving {file_name}: {str(e)}"

In [13]:
def save_in_batches(df, batch_size=5000):
    """
    Function to save content in batches to .txt files.
    """
    num_rows = len(df)
    num_batches = (num_rows // batch_size) + (1 if num_rows % batch_size != 0 else 0)

    for i in range(num_batches):
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, num_rows)
        
        batch_df = df.iloc[start_index:end_index]
        
        # tqdm progress bar for saving
        with tqdm(total=len(batch_df), desc=f"Saving batch {i}") as pbar:
            for index, row in batch_df.iterrows():
                save_content_to_txt(row)
                pbar.update(1)
                

In [14]:
# Call the function to save in batches
save_in_batches(df)

Saving batch 16: 100%|█████████████████████████████████████████████████████████████| 2498/2498 [00:53<00:00, 46.44it/s]


In [15]:
conn.close()